In [2]:
import findspark
findspark.init()
from pyspark import SparkConf , SparkContext
from pyspark.sql import SparkSession , SQLContext
import requests
import json
from bs4 import BeautifulSoup
from pymongo import MongoClient
import csv



In [4]:
conf = SparkConf().setMaster('local').setAppName('MyApp')
spark  = SparkSession.builder.config(conf=conf).getOrCreate()

In [12]:
def crawler_inside(link):
    #print (data_temp['Title'])
    page_inside = requests.get(link).text
    soup_inside  = BeautifulSoup(page_inside , "html.parser")
    
    if soup_inside.find("div" , {"class":"body-container"}) != None:
        content_inside = soup_inside.find("div" , {"class":"body-container"}).text
    else:
        content_inside = 'missing'
        print('missing')
    #print (content_inside)
    
    if soup_inside.find_all("img" , {"class":"slider__frame glide__slide"}) != None:
        image_urls = []
        image_urls_raw  =soup_inside.find_all("img" , {"class":"slider__frame glide__slide"})
        for image_url in image_urls_raw:
            image_urls.append(image_url['src'])
    else:
        image_urls = 'missing'
        print('missing')
    
    
   
    if soup_inside.find_all("div" , {"class":"tech-item"}) != None:
        additional_information = []
        additional_information_raw = soup_inside.find_all("div" , {"class":"tech-item"})
        for info in additional_information_raw:
            if info != None:
                additional_information.append(info.text)
            else:
                additional_information.append('missing')
    else:
        additional_information = 'missing'
        print('missing')

        
    
    
    
    return (content_inside , image_urls ,additional_information )

In [13]:
def crawler_outsite(urls):
    #client = MongoClient("192.168.1.148",27017)
    #db=client.PHIVU
    data_final = []
    i = 0
    for url in urls:
        print (url)
        page  = requests.get(url).text
        #print(page)
        soup = BeautifulSoup(page, "html.parser")
        item_list = soup.find('div' , {"id": "list-box"}).find_all("div",{"class":"list-item-container"})
        for item in item_list:
            i = i+1
            data_temp = []
            data_temp.append(i)
            #link = item.find("a",{"class":"list-item__link"}).text
            #print (link['href'])
            #link_list = [a['href'] for a in item.find_all('a', href=True)]
            data_temp.append(item.find("h2" , {"class":"list-item__title"}).text)
            if item.find("span", {"class":"list-item__price"}) != None:
                data_temp.append(item.find("span", {"class":"list-item__price"}).text)
            else:
                data_temp.append('missing')
            if item.find("span", {"class":"list-item__area"}) != None :
                data_temp.append(item.find("span", {"class":"list-item__area"}).find('b').text)
            else :
                data_temp.append('missing')
            #print (data_temp['Area'])
            if item.find("span", {"class":"list-item__location"}) != None:
                data_temp.append(item.find("span", {"class":"list-item__location"}).text)
            else:
                data_temp.append('missing')
            if  item.find("span", {"class":"list-item__date"}) != None:
                data_temp.append(item.find("span", {"class":"list-item__date"}).text)
            else:
                data_temp.append('missing')
            link = item.find('a' , href = True)['href']
            inside_result = crawler_inside(link)
            #print ("I'm here")
            data_temp.append(inside_result[0])
            #data_temp["Image URLs"] = inside_result[1]
            data_temp.append(inside_result[2])
            f = open('muaban.csv', 'a')
            writer = csv.writer(f)
            writer.writerow(data_temp)

            #print (data_temp['Image URLs'])
            #data_temp_json = (json.dumps(data_temp, ensure_ascii=False).encode('utf8')).decode()
            #db.data_mubannet.insert_one(data_temp)
            data_final.append(data_temp)
            #data_final["Object " + str(i) ] = data_temp
    #data_final_json = (json.dumps(data_final, ensure_ascii=False).encode('utf8')).decode()
    return data_final

In [14]:
urls  =  ["https://muaban.net/ban-can-ho-chung-cu-tap-the-toan-quoc-l0-c38?sort=1"]
for x in range (2,501):
    urls.append("https://muaban.net/ban-can-ho-chung-cu-tap-the-toan-quoc-l0-c38?sort=1&cp="+str(x))
#f = open("crawler_result.json" , "w" ,encoding='utf8')
#f.write(crawl_outsite(urls))
#f.close()
#print (crawl_outsite(urls))
#df = spark.createDataFrame(crawler_outsite(urls))

crawler_outsite(urls)
#df.show(1)
#for x in test:
#    print (x)

https://muaban.net/ban-can-ho-chung-cu-tap-the-toan-quoc-l0-c38?sort=1


[[1,
  'Bán căn hộ Ehome4- Nội thất Full đẹp- 849tr ',
  '849.000.000 đ',
  'missing',
  '\n\n\n Thị xã Thuận An, Bình Dương\n',
  '\n\n\n 17/04/2020\n',
  '\n* Thông tin căn hộ bán: 849triệu (TL).- Thuộc KDC EHome 4 - Thuận An - Bình Dương. Vị trí căn hộ góc, view thoáng mát, sổ hồng riêng sở hữu vĩnh viễn.+ Giá bán: 849triệu(TL).Nội thất gồm: Máy lạnh 1.5HP, máy giặt toshiba, máy NC nóng, bộ sofa, giường ngủ, tủ áo gỗ, tủ bếp trên dưới, bộ bếp gas, rèm cửa... Nhà thực tế vui lòng xem hình đính kèm.- Quý khách quan tâm vui lòng LH chính chủ: 090 439 16 76- Hỗ trợ hồ sơ vay ngân hàng lên đến 550triệu khi quý khách có nhu cầu, lãi suất ưu đãi. Chỉ cần thanh toán trước vốn tự có, số còn lại ngân hàng sẽ hỗ trợ vay vốn.- Tiện ích ngoại khu: Gần Aeon Mall, bệnh viện Hạnh Phúc, chợ Lái Thiêu, giáp ranh Q12, cách ngã tư Bình Phước 2km.- Tiện ích nội khu: Hồ bơi, trường học, công viên, siêu thị Family Mart.- Trân trọng!\n',
  ['\nĐịa chỉ:\nQuốc Lộ 13 Phường Vĩnh Phú\n',
   '\nTầng/Lầu:\n3\n',